In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [15]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.optimizers import Adam


VOCAB_SIZE = 5000
MAXLEN = 25

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('/kaggle/input/fake-news/train.csv')
test = pd.read_csv('/kaggle/input/fake-news/test.csv')

train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

## Data Preprocessing and cleaning

In [8]:
train = train.fillna(' ')
test = test.fillna(' ')

In [11]:
train.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [12]:
train['Title and Author'] = train['title'] + ' ' + train['author']
test['Title and Author'] = test['title'] + ' ' + test['author']

train.head()

,id,title,author,text,label,Title and Author
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [13]:
X = train.drop(['id','label'], axis = 1)
y = train.label

X.shape, y.shape

((20800, 4), (20800,))

In [17]:
ps = PorterStemmer()
corpus = []

In [19]:
for i in range(len(X)):
    review = re.sub('[^a-zA-Z]', ' ', X['Title and Author'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
test_corpus = []
for i in range(len(test)):
    review = re.sub('[^a-zA-Z]', ' ', test['Title and Author'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    test_corpus.append(review)

**One Hot Representation**

In [21]:
onehot_rep = [one_hot(word, VOCAB_SIZE) for word in corpus]
onehot_rep_test = [one_hot(word, VOCAB_SIZE) for word in test_corpus]

**Padding**

In [23]:
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen = MAXLEN)
embedded_docsTest = pad_sequences(onehot_rep_test, padding='pre', maxlen = MAXLEN)

## Creating and Training Model

In [25]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 40, input_length=MAXLEN))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [26]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

test_final = np.array(embedded_docsTest)

In [27]:
model.fit(X_final, y_final, epochs=20, batch_size=64)

Epoch 1/20
325/325 [==============================] - 11s 28ms/step - loss: 0.3250 - accuracy: 0.8297
Epoch 2/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0236 - accuracy: 0.9942
Epoch 3/20
325/325 [==============================] - 9s 29ms/step - loss: 0.0091 - accuracy: 0.9974
Epoch 4/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0049 - accuracy: 0.9986
Epoch 5/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0043 - accuracy: 0.9988
Epoch 6/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0041 - accuracy: 0.9987
Epoch 7/20
325/325 [==============================] - 9s 29ms/step - loss: 0.0018 - accuracy: 0.9993
Epoch 8/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0026 - accuracy: 0.9991
Epoch 9/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0017 - accuracy: 0.9993
Epoch 10/20
325/325 [==============================] - 9s 28ms/step - loss: 0.0014 - accur

In [43]:
submission = pd.read_csv('/kaggle/input/fake-news/submit.csv')
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [44]:
submission['id'] = test['id']
submission['label'] = model.predict_classes(test_final)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [45]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [46]:
submission.to_csv('submit.csv', index=False)